<a href="https://colab.research.google.com/github/l-87hjl/3i-atlas-public-data/blob/main/horizons_works-batch_fetcher_3i_atlas_ipynb_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ⚠️ Pipeline Context

This notebook implements **Pipeline B**: API direct-fetch against the **current JPL solution**.

**Key limitation**: API queries always use the current orbital solution. You cannot query historical solutions like Sol44.

---

**Rotated Error Ellipse Methodology**: This notebook fetches SMAA, SMIA, THETA parameters. Do NOT treat RA/DEC uncertainties as independent. See `documentation/SIGMA_UNCERTAINTY_INTERPRETATION_JPL_HORIZONS.txt`.

# 🌌 Horizons API Batch Fetcher for Comet 3I/ATLAS

This notebook automatically fetches ephemeris data for observations of comet 3I/ATLAS from JPL Horizons.

**Features:**
- Handles variable data formats including optional flags
- Extracts rotated error ellipse parameters (SMAA, SMIA, THETA)
- Robust parsing of RA/DEC coordinates
- Automatic retry logic with rate limiting

## 📋 Instructions

1. **Upload your CSV file**: Click the folder icon on the left → Upload → Select `observations-timestamp-observatory-only.csv`
2. **Run all cells**: Click `Runtime` → `Run all` (or press Ctrl+F9)
3. **Download results**: The final cell will let you download your results CSV

## 📊 Data Extracted

For each observation, we extract:
- UTC time
- RA (ICRF)
- DEC (ICRF)
- dRA*cosD (angular rate in arcsec/hour)
- d(DEC)/dt (angular rate in arcsec/hour)
- RA_3sigma (uncertainty in arcseconds)
- DEC_3sigma (uncertainty in arcseconds)
- SMAA_3sig (error ellipse semi-major axis in arcseconds)
- SMIA_3sig (error ellipse semi-minor axis in arcseconds)
- Theta (error ellipse orientation in degrees, clockwise from +RA toward +DEC)

---

## Step 1: Install Required Libraries

In [1]:
# Install/verify requests library (usually already available in Colab)
!pip install -q requests
print("✓ Libraries ready!")

✓ Libraries ready!


## Step 2: Import Libraries and Define Functions

In [2]:
import csv
import re
import time
import requests
from typing import Dict, List, Optional
from urllib.parse import urlencode
from google.colab import files
import io

print("✓ Imports successful!")

✓ Imports successful!


In [3]:
def build_horizons_url(timestamp: str, observatory: str) -> str:
    """
    Build Horizons API URL for given timestamp and observatory.

    Parameters:
    -----------
    timestamp : str
        ISO format timestamp (YYYY-MM-DD HH:MM:SS)
    observatory : str
        MPC observatory code

    Returns:
    --------
    str : Full API URL
    """
    params = {
        'format': 'text',
        'COMMAND': "'DES=1004083;'",  # 3I/ATLAS designation
        'MAKE_EPHEM': "'YES'",
        'EPHEM_TYPE': "'OBSERVER'",
        'CENTER': f"'{observatory}'",
        'TLIST': f"'{timestamp}'",
        'QUANTITIES': "'1,3,36,37'",  # Astrometry + rates + uncertainties + error ellipse
        'EXTRA_PREC': "'YES'",
        'TIME_DIGITS': "'SECONDS'",
        'CSV_FORMAT': "'NO'"
    }

    base_url = "https://ssd.jpl.nasa.gov/api/horizons.api"
    return f"{base_url}?{urlencode(params)}"


def parse_horizons_response(response_text: str, timestamp: str, observatory: str) -> Optional[Dict[str, str]]:
    """
    Parse Horizons API response to extract required fields.

    Uses regex to find coordinates directly, ignoring AM/PM markers and handling
    variable data formats including optional observation flags.

    Parameters:
    -----------
    response_text : str
        Full Horizons API response
    timestamp : str
        Original timestamp for error reporting
    observatory : str
        Observatory code for error reporting

    Returns:
    --------
    Optional[Dict[str, str]] : Parsed data or None if parsing fails
    """
    # Find the data line between $$SOE and $$EOE markers
    soe_pattern = r'\$\$SOE\s+(.*?)\s+\$\$EOE'
    match = re.search(soe_pattern, response_text, re.DOTALL)

    if not match:
        print(f"    ⚠️ No data found for {timestamp} at {observatory}")
        return None

    data_line = match.group(1).strip()

    # Use regex to find coordinate patterns directly
    # RA format: HH MM SS.decimal (with spaces)
    # Dec format: +/-DD MM SS.decimal (with spaces)
    coord_pattern = r'(\d{1,2})\s+(\d{1,2})\s+(\d{1,2}\.\d+)'

    coords = list(re.finditer(coord_pattern, data_line))

    if len(coords) < 2:
        print(f"    ⚠️ Could not find RA/Dec coordinates for {timestamp} at {observatory}")
        return None

    # First coordinate is RA
    ra_match = coords[0]
    ra_h, ra_m, ra_s = ra_match.groups()
    ra_icrf = f"{ra_h} {ra_m} {ra_s}"

    # Second coordinate is Dec - look backwards for sign
    dec_match = coords[1]
    dec_start = dec_match.start()
    dec_sign = ''
    for j in range(dec_start - 1, max(0, dec_start - 5), -1):
        if data_line[j] in ['+', '-']:
            dec_sign = data_line[j]
            break

    dec_d, dec_m, dec_s = dec_match.groups()
    dec_icrf = f"{dec_sign}{dec_d} {dec_m} {dec_s}"

    # Now split to get the numeric values after coordinates
    parts = data_line.split()

    # Find Dec degrees with sign in the parts list
    dec_deg_with_sign = f"{dec_sign}{dec_d}"
    try:
        dec_idx = parts.index(dec_deg_with_sign)
    except ValueError:
        # Sometimes the sign might be separate
        try:
            dec_idx = parts.index(dec_d)
        except ValueError:
            print(f"    ⚠️ Could not locate Dec in parts list")
            return None

    # After Dec (deg, min, sec), we have: dRA*cosD, d(DEC)/dt, RA_3sig, DEC_3sig, SMAA, SMIA, Theta
    dra_idx = dec_idx + 3
    ddec_idx = dec_idx + 4
    ra_sig_idx = dec_idx + 5
    dec_sig_idx = dec_idx + 6
    smaa_idx = dec_idx + 7
    smia_idx = dec_idx + 8
    theta_idx = dec_idx + 9

    # Check if we have enough fields
    if len(parts) < theta_idx + 1:
        print(f"    ⚠️ Insufficient data fields for {timestamp} at {observatory}")
        print(f"       Expected {theta_idx + 1} fields, found {len(parts)}")
        return None

    # Extract UTC time - everything before the RA starts
    time_end_idx = data_line.index(ra_h)
    utc_time = data_line[:time_end_idx].strip()
    # Remove AM/PM markers if present
    utc_time = re.sub(r'\s*[AP]m?\s*$', '', utc_time, flags=re.IGNORECASE)

    result = {
        'timestamp': timestamp,
        'observatory': observatory,
        'utc_time': utc_time,
        'ra_icrf': ra_icrf,
        'dec_icrf': dec_icrf,
        'dra_cosd': parts[dra_idx],
        'ddec_dt': parts[ddec_idx],
        'ra_3sigma': parts[ra_sig_idx],
        'dec_3sigma': parts[dec_sig_idx],
        'smaa_3sig': parts[smaa_idx],
        'smia_3sig': parts[smia_idx],
        'theta': parts[theta_idx]
    }

    return result


def fetch_horizons_data(url: str, max_retries: int = 3) -> Optional[str]:
    """
    Fetch data from Horizons API with retry logic.

    Parameters:
    -----------
    url : str
        Full API URL
    max_retries : int
        Maximum number of retry attempts (default: 3)

    Returns:
    --------
    Optional[str] : Response text or None if all retries failed
    """
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=30)
            if response.status_code == 200:
                return response.text
            else:
                print(f"      Attempt {attempt + 1}/{max_retries}: HTTP {response.status_code}")
        except Exception as e:
            print(f"      Attempt {attempt + 1}/{max_retries} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)  # Wait before retry

    return None


print("✓ Functions defined!")

✓ Functions defined!


## Step 3: Upload Your CSV File

Run this cell and select your `observations-timestamp-observatory-only.csv` file.

In [7]:
# Upload the CSV file
print("📁 Please select your CSV file...")
uploaded = files.upload()

# Get the filename
csv_filename = list(uploaded.keys())[0]
print(f"\n✓ Uploaded: {csv_filename}")

📁 Please select your CSV file...


Saving observations_timestamp_observatory_only_20251217_20260102_v4a7.csv to observations_timestamp_observatory_only_20251217_20260102_v4a7.csv

✓ Uploaded: observations_timestamp_observatory_only_20251217_20260102_v4a7.csv


## Step 4: Load and Preview Observations

In [8]:
# Read observations from CSV
observations = []
with open(csv_filename, 'r') as f:
    reader = csv.DictReader(f)
    observations = list(reader)

print(f"📊 Loaded {len(observations)} observations\n")
print("First 5 observations:")
for i, obs in enumerate(observations[:5], 1):
    print(f"  {i}. {obs['timestamp']} at observatory {obs['observatory']}")

if len(observations) > 5:
    print(f"  ... and {len(observations) - 5} more")

📊 Loaded 193 observations

First 5 observations:
  1. 2025-12-17 00:09:07.776000 at observatory L16
  2. 2025-12-17 00:39:28.224000 at observatory L16
  3. 2025-12-17 01:09:51.264000 at observatory L16
  4. 2025-12-17 01:45:20.505600 at observatory Z92
  5. 2025-12-17 01:56:31.488000 at observatory Z92
  ... and 188 more


## Step 5: Fetch and Parse All Data

This will take a few minutes. Progress will be shown below.

**Note**: JPL Horizons has rate limiting. We add a small delay between requests to be respectful of their server.

In [9]:
results = []
errors = []
total = len(observations)

print("🚀 Starting data collection...\n")
print("=" * 80)

for i, obs in enumerate(observations, 1):
    timestamp = obs['timestamp']
    observatory = obs['observatory']

    print(f"\n[{i}/{total}] {timestamp} at {observatory}")
    print("    Fetching...", end=' ')

    # Build URL and fetch data
    url = build_horizons_url(timestamp, observatory)
    response_text = fetch_horizons_data(url)

    if response_text is None:
        print("❌ FAILED - Could not fetch data")
        errors.append({
            'timestamp': timestamp,
            'observatory': observatory,
            'error': 'Failed to fetch data from API'
        })
        continue

    # Parse the response
    parsed = parse_horizons_response(response_text, timestamp, observatory)

    if parsed is None:
        print("❌ FAILED - Could not parse response")
        errors.append({
            'timestamp': timestamp,
            'observatory': observatory,
            'error': 'Failed to parse API response'
        })
    else:
        print("✅ SUCCESS")
        print(f"    RA: {parsed['ra_icrf']}, DEC: {parsed['dec_icrf']}")
        results.append(parsed)

    # Rate limiting - be nice to JPL servers
    if i < total:
        time.sleep(0.5)  # Half second between requests

print("\n" + "=" * 80)
print(f"\n✓ Complete! Successfully processed {len(results)}/{total} observations")
if errors:
    print(f"⚠️  {len(errors)} observations failed")

🚀 Starting data collection...


[1/193] 2025-12-17 00:09:07.776000 at L16
    Fetching... ✅ SUCCESS
    RA: 10 56 26.723861, DEC: +06 13 01.28851

[2/193] 2025-12-17 00:39:28.224000 at L16
    Fetching... ✅ SUCCESS
    RA: 10 56 20.930248, DEC: +06 13 31.04241

[3/193] 2025-12-17 01:09:51.264000 at L16
    Fetching... ✅ SUCCESS
    RA: 10 56 15.124310, DEC: +06 14 00.83669

[4/193] 2025-12-17 01:45:20.505600 at Z92
    Fetching... ✅ SUCCESS
    RA: 10 56 08.393286, DEC: +06 14 35.01058

[5/193] 2025-12-17 01:56:31.488000 at Z92
    Fetching... ✅ SUCCESS
    RA: 10 56 06.258514, DEC: +06 14 45.97909

[6/193] 2025-12-17 02:06:59.097600 at Z92
    Fetching... ✅ SUCCESS
    RA: 10 56 04.261246, DEC: +06 14 56.23850

[7/193] 2025-12-17 03:47:21.004800 at 970
    Fetching... ✅ SUCCESS
    RA: 10 55 45.073045, DEC: +06 16 34.79465

[8/193] 2025-12-17 04:06:25.027200 at 970
    Fetching... ✅ SUCCESS
    RA: 10 55 41.423699, DEC: +06 16 53.49110

[9/193] 2025-12-17 04:26:36.009600 at 970
    Fe

## Step 6: Display Summary Statistics

In [10]:
if results:
    print("📈 Summary Statistics:")
    print(f"   Total observations: {total}")
    print(f"   Successful: {len(results)}")
    print(f"   Failed: {len(errors)}")
    print(f"   Success rate: {len(results)/total*100:.1f}%")

    # Show unique observatories
    observatories = set(r['observatory'] for r in results)
    print(f"\n   Unique observatories: {len(observatories)}")
    print(f"   Observatory codes: {', '.join(sorted(observatories))}")

    # Show time range
    timestamps = [r['timestamp'] for r in results]
    print(f"\n   Time range: {min(timestamps)} to {max(timestamps)}")
else:
    print("⚠️ No results to display")

📈 Summary Statistics:
   Total observations: 193
   Successful: 193
   Failed: 0
   Success rate: 100.0%

   Unique observatories: 40
   Observatory codes: 152, 213, 215, 290, 703, 900, 970, B67, B72, B74, C23, C40, C82, C94, D68, D69, G05, H78, I81, J13, L16, L85, L92, M09, M57, M73, O56, Q14, Q21, Q23, R17, U76, U94, V16, V21, W50, X08, Y87, Y88, Z92

   Time range: 2025-12-17 00:09:07.776000 to 2025-12-23 03:25:15.888000


## Step 7: Save Results to CSV

In [11]:
if results:
    output_filename = 'horizons_results_3i_atlas.csv'

    fieldnames = [
        'timestamp', 'observatory', 'utc_time',
        'ra_icrf', 'dec_icrf',
        'dra_cosd', 'ddec_dt',
        'ra_3sigma', 'dec_3sigma',
        'smaa_3sig', 'smia_3sig', 'theta'
    ]

    # Write to CSV
    with open(output_filename, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)

    print(f"✓ Results saved to {output_filename}")
    print(f"\n📥 Downloading...")
    files.download(output_filename)
    print("✓ Download complete!")
else:
    print("⚠️ No results to save")

✓ Results saved to horizons_results_3i_atlas.csv

📥 Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Download complete!


## Step 8: Save Error Log (if any errors occurred)

In [12]:
if errors:
    error_filename = 'horizons_error_log.txt'

    with open(error_filename, 'w') as f:
        f.write("=" * 70 + "\n")
        f.write("HORIZONS BATCH FETCHER ERROR LOG\n")
        f.write("=" * 70 + "\n\n")
        f.write(f"Total Observations: {total}\n")
        f.write(f"Successful: {len(results)}\n")
        f.write(f"Failed: {len(errors)}\n\n")
        f.write("=" * 70 + "\n")
        f.write("DETAILED ERROR LIST\n")
        f.write("=" * 70 + "\n\n")

        for i, error in enumerate(errors, 1):
            f.write(f"Error #{i}\n")
            f.write(f"  Timestamp: {error['timestamp']}\n")
            f.write(f"  Observatory: {error['observatory']}\n")
            f.write(f"  Error: {error['error']}\n")
            f.write("-" * 70 + "\n\n")

    print(f"⚠️ {len(errors)} errors occurred")
    print(f"✓ Error log saved to {error_filename}")
    print(f"\n📥 Downloading error log...")
    files.download(error_filename)
    print("✓ Download complete!")
else:
    print("✓ No errors - all observations processed successfully!")

✓ No errors - all observations processed successfully!


## 🎉 All Done!

Your results have been downloaded:
- `horizons_results_3i_atlas.csv` - Your compiled ephemeris data
- `horizons_error_log.txt` - Error log (if any failures occurred)

## ⚠️ Important Notes on Uncertainty Interpretation

The uncertainty values in this data represent **formal orbit covariance uncertainties**, not observational measurement error.

**Do NOT treat RA_3sigma and DEC_3sigma as independent axis-aligned confidence bounds.**

For proper statistical analysis of residuals:
1. Use the rotated error ellipse defined by SMAA_3sig, SMIA_3sig, and THETA
2. Rotate residual vectors by THETA angle
3. Normalize by the ellipse axes
4. Test whether the normalized distance < 1.0 (inside 3σ boundary)

See the project documentation files:
- `EPHEMERIS_FIELD_SPEC_3I_ATLAS.txt`
- `SIGMA_UNCERTAINTY_INTERPRETATION_JPL_HORIZONS.txt`

---

You can now use this data for your orbital refinement analysis!